In [4]:
from __future__ import print_function, division

%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_transforms = {
    'carspotting_batch': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'carspotting': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #transforms.ToPILImage(),
    ]),
    'to_tensor': transforms.Compose([
        transforms.ToTensor()
    ])
}

In [6]:
#print(model_conv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from skimage import io
from PIL import Image
import glob

import glob
from shutil import copyfile
import os

model_conv = models.vgg19(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
#model_conv.to(device)

#allowed_indices = [436, 705, 751, 817, 864, 627, 734, 468]
allowed_indices = set([751, 817, 864, 627, 734, 468])
print(allowed_indices)
transform = data_transforms['carspotting']

carpics= glob.glob('./data/**/*.jpg', recursive=True)
trashdir = './cleaned_data/trash/'
cardir = './cleaned_data/car/'

if not os.path.exists(trashdir):
    os.makedirs(trashdir)
    
if not os.path.exists(cardir):
    os.makedirs(cardir)

{864, 751, 817, 627, 468, 734}


In [21]:
batch_size = 100
steps = len(carpics) // batch_size
count = 0

for i in range(steps+1):
    if i != steps:
        image_batch = carpics[i*batch_size: (i+1)*batch_size]
    elif i == steps:
        image_batch = carpics[i*batch_size: len(carpics)]
    
    #transform = data_transforms['carspotting_batch']
    #input_batch = np.zeros((len(image_batch), 3, 224, 224), dtype=np.float64)
    input_batch = []
    for i, image_path in enumerate(image_batch):
        pil_image = Image.open(image_path)
        transformed_image = transform(pil_image)
        transformed_image.unsqueeze_(0)
        import pdb; pdb.set_trace()
        b, c, h, w = transformed_image.shape
        if not (c == 3 and h == 224 and w == 224):
            continue
        input_batch.append(transformed_image)
        #import pdb; pdb.set_trace()
        #input_batch[i, :, :, :] = transformed_image_tensor
    if len(input_batch) == 0:
        continue
    input = torch.cat(input_batch)
    #input.to(device)
    #import pdb; pdb.set_trace()
        
    try:
        output = model_conv(input)
    except RuntimeError:
        continue
        
    #import pdb; pdb.set_trace()
        
    sorted_top_5_labels = np.argsort(output)
    sorted_top_5_labels = sorted_top_5_labels[:, -5:]
    
    for i, image in enumerate(image_batch):
        trash = False
        car = False
        
        #import pdb; pdb.set_trace()

        if len(set(sorted_top_5_labels[i, :].numpy()).intersection(allowed_indices)) != 0:
            car = True
        else:
            trash = True
            
        file = image

        if trash:
            outDir = trashdir + '/'.join(file.split('/')[2:-1])
            if not os.path.exists(outDir):
                os.makedirs(outDir)

            copyfile(file, trashdir+'/'.join(file.split('/')[2:]))

        if car:
            outDir = cardir + '/'.join(file.split('/')[2:-1])
            if not os.path.exists(outDir):
                os.makedirs(outDir)

            copyfile(file, cardir+'/'.join(file.split('/')[2:]))
    
    count += 1
    print(count*batch_size)
        
"""
for i in range(len(carpics)):
    import pdb; pdb.set_trace()
    image_path = carpics[i]
    pil_image = Image.open(image_path)
    transformed_image = transform(pil_image)
    transformed_image.unsqueeze_(0)
    
    try:
        output = model_conv(transformed_image)
    except RuntimeError:
        continue
    
    sorted_top_5_labels = np.argsort(output)[:, ::-1][:5]
    
    trash = False
    car = False
    
    if len(set(sorted_top_5_labels).intersection(allowed_indices)) != 0:
        car = True
    else:
        trash = True
        
    import pdb; pdb.set_trace()
        
    if trash:
        outDir = trashdir + '/'.join(file.split('/')[3:-1])
        if not os.path.exists(outDir):
            os.makedirs(outDir)

        copyfile(file, trashdir+'/'.join(file.split('/')[3:]))

    if car:
        outDir = cardir + '/'.join(file.split('/')[3:-1])
        if not os.path.exists(outDir):
            os.makedirs(outDir)

        copyfile(file, cardir+'/'.join(file.split('/')[3:]))
        
    if i % 50 == 0:
        print(i)
"""

> <ipython-input-21-7d1c0fc7bdce>(19)<module>()
-> b, c, h, w = transformed_image.shape
> <ipython-input-21-7d1c0fc7bdce>(20)<module>()
-> if c != 3:
> <ipython-input-21-7d1c0fc7bdce>(18)<module>()
-> import pdb; pdb.set_trace()
torch.Size([1, 3, 224, 224])
*** SyntaxError: invalid syntax
> <ipython-input-21-7d1c0fc7bdce>(19)<module>()
-> b, c, h, w = transformed_image.shape
1
3
224
224


BdbQuit: 

In [7]:
#print(model_conv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from skimage import io
from PIL import Image
import glob

import glob
from shutil import copyfile
import os

cardir = './cleaned_data/car/'
audidir = './cleaned_data/audi/'

carpics= glob.glob('./cleaned_data/car/**/*.jpg', recursive=True)

for file in carpics:
    if "audi" in str(file).lower():
        import pdb; pdb.set_trace()
        outDir = audidir + '/'.join(file.split('/')[2:-1])
        if not os.path.exists(outDir):
            os.makedirs(outDir)

        copyfile(file, audidir+'/'.join(file.split('/')[2:]))